In [1]:
import numpy as np


In [1]:
"""
Class for running genetic algorithms
"""
# genetic algorithm search of the test coverage problem
from numpy.random import randint
from numpy.random import rand

class TestDatabase:
    # This class represents our test database
    # It is already implemented, no need to change it if you don't want

    def __init__(self):
        self.coverage=[]
        self.time=[]

    def get_time(self,i):
        return self.time[i]

    def get_coverage(self,i):
        return self.coverage[i]

    def get_number_of_tests(self):
        return len(self.time)

    def init_random(self,n_tests, max_time, max_code, p=0.05):
        # initialize a test database randomly
        self.coverage = []
        self.time = []

        for i in range(n_tests):
            t_time= randint(1,max_time)
            t_coverage=[]
            for i in range(1,max_code+1):
                if rand()<p:
                    t_coverage.append(i)
            self.time.append(t_time)
            self.coverage.append(t_coverage)


    def load_from_file(self,fn):
        # load test database from file with name fn
        # file format:
        #  line i represents test i as comma separated value
        #   the first value is the time to execute the test
        #   the other values are the lines covered by the test
        #  Example
        #   5, 1, 5, 7   -> A test that covers lines 1,5,7 and takes 5 seconds to run

        self.coverage = []
        self.time = []
        with open(fn,'rt') as fd:
            for line in fd.readlines():
                words=line.split(",")
                self.time.append(float(words[0]))
                self.coverage.append(list(map(lambda x: int(x), words[1:])))

    def write_to_file(self,fn):
        # write the test database to a file with name fn
        with open(fn, 'wt') as fd:
            for time, coverage in zip(self.time, self.coverage):
                fd.write(str(time))
                for i in coverage:
                    fd.write(", "+str(i))
                fd.write("\n")



In [60]:
import random

# fitness function, it is ready, no need to change it
def fitness(x, db, max_time):
    coverage  = set()
    total_time = 0

    for i in range(len(x)):
        if x[i] == 1:
            coverage = coverage.union( db.get_coverage(i))
            total_time = total_time + db.get_time(i)

    if total_time <= max_time:
        return len(coverage)
    else:
        return 0

def generate_custom_distribution(scores):
    total_score = sum(scores)
    probabilities = [sc / total_score for sc in scores]
    custom_distribution = {}
    for i, probability in enumerate(probabilities):
        custom_distribution[i] = probability
    return custom_distribution

def randomize_integer(custom_distribution):
    # Extract the values and probabilities from the custom distribution
    values = list(custom_distribution.keys())
    probabilities = list(custom_distribution.values())

    # Generate a random value based on the custom distribution
    randomized_value = random.choices(values, probabilities)[0]

    return randomized_value


# selection
def selection(population, scores):
    assert min(scores) >= 0, "All scores must be positive or 0! {}".format(scores)

    # TODO implement a (better) selection function
    # proportional fitness implementation
    # generate random distribution according to scores
    custom_distribution = generate_custom_distribution(scores)
    # random the individual to select
    selected_idx = randomize_integer(custom_distribution)

    # Here we just select an individual at random
    # this is a bad strategy... you should implement something better
    # selection_ix = randint(len(population))

    return population[selected_idx]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # TODO implement a (better) crossover function

    # children are copies of parents by default
    assert len(p1)-1 > 1, "the length of string is too short for crossover!"
    c1, c2 = p1.copy(), p2.copy() # keep the copy for no else
    if random.random() < r_cross: # crossover accepted
        # pick a point in length of children to cross over should be from index 1 till length -1
        cross_idx = random.randint(1, len(c1)-1) # note: randint is inclusive
        # crossover
        c1 = p1[:cross_idx] + p2[cross_idx:]
        c2 = p2[:cross_idx] + p1[cross_idx:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    # TODO implement a (better) mutation function
    for i in range(len(bitstring)):
        if random.random() < r_mut:
            bitstring[i] = 0 if bitstring[i] == 1 else 1
    return bitstring

# genetic algorithm
# This is ready, no need to change it if you don't want
def genetic_algorithm(db, maxtime, fitness, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    population = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, fitness(population[0],db,maxtime)

    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [fitness(c,db,maxtime ) for c in population]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = population[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  population[i], scores[i]))
        # select parents
        selected = [selection(population, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        population = children
    return [best, best_eval]


In [77]:

# intialize the problem
db= TestDatabase()
db.load_from_file("problem1.txt")
max_time=1000

# define the total iterations, you can change this
n_iter = 100000
# bits: one bit per each test that may be executed
n_bits =  db.get_number_of_tests()
# define the population size, you can change this
n_pop = 100
# crossover rate, you can change this
r_cross = 0.9
# mutation rate, you can change this
r_mut = 1.0 / float(n_bits)

# perform the genetic algorithm search
best, score = genetic_algorithm(db, max_time, fitness, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
print('f(%s) = %f' % (best, score))

>0, new best f([1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0]) = 648.000
>0, new best f([0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1]) = 734.000
>0, new best f([0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]) = 735.000
>1, new best f([0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1]) = 750.000
>1, new best f([1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]) = 770.000
>8, new best f([0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 

In [37]:
"""
max_time=1000

# define the total iterations, you can change this
n_iter = 10000
# bits: one bit per each test that may be executed
n_bits =  db.get_number_of_tests()
# define the population size, you can change this
n_pop = 100
# crossover rate, you can change this
r_cross = 0.9
# mutation rate, you can change this
r_mut = 1.0 / float(n_bits)

f([0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1]) = 843.000000



"""

2


[[1, 1, 1, 0], [1, 0, 1, 1]]